In [1]:
# Load libraries
import numpy as np

In [2]:
# Set seed
np.random.seed(10)

In [3]:
# Doc params
V = 50
N = 300
K = 3
M = 2

In [4]:
# Set true params
alpha_true = np.array([10, 1, 2])
beta_true = np.random.randint(1, 10, V)

In [5]:
# Generate data
phi_true = np.zeros((K, V))
for k in range(K):
    phi_true[k, :] = np.random.dirichlet(beta_true)

In [6]:
theta_true = np.zeros((M, K))
for m in range(M):
    theta_true[m,:] = np.random.dirichlet(alpha_true)

In [7]:
doc_lens = np.random.randint(100, N, M)
z_true = {}
w = {}
for m in range(M):
    z_true[m] = []
    w[m] = []
    for n in range(doc_lens[m]):
        z_true[m].extend(np.nonzero(np.random.multinomial(1, theta_true[m,:]))[0])
        w[m].extend(np.nonzero(np.random.multinomial(1, phi_true[z_true[m][n], :]))[0])

In [8]:
#z_true = np.zeros((M, N), dtype = "int")
#w = np.zeros((M, N), dtype = "int")
#for m in range(M):
#    for n in range(N):
#        z_true[m, n] = np.nonzero(np.random.multinomial(1, theta_true[m,:]))[0]
#        w[m, n] = np.nonzero(np.random.multinomial(1, phi_true[z_true[m, n], :]))[0]

# phi_true[z_true[m][n]]

In [9]:
# Set initial z randomly
z = {}
for m in range(M):
    z[m] = []
    for n in range(doc_lens[m]):
        z[m].extend(np.nonzero(np.random.multinomial(1, np.ones(K)/K))[0])

In [10]:
# Create count matrices
A = np.zeros((M, K))
for m in range(M):
    for k in range(K):
        A[m, k] = sum(np.array(z[m]) == k)

In [11]:
B = np.zeros((K, V))
for m in range(M):
    for n in range(doc_lens[m]):
        B[z[m][n], w[m][n]] += 1

In [12]:
C = np.zeros(K)
for m in range(M):
    for n in range(doc_lens[m]):
        C[z[m][n]] += 1

In [13]:
# set hyperparameters alpha and beta
alpha = np.ones(K)
beta =np.ones(V)

In [14]:
# start gibbs sampler
num_iter = 10000
for i in range(num_iter):
    for m in range(M):
        for n in range(doc_lens[m]):
            A[m, z[m][n]] -= 1
            B[z[m][n], w[m][n]] -= 1
            C[z[m][n]] -= 1
            p = np.zeros(K)
            for k in range(K):
                p[k] = (A[m, k] + alpha[k])*((B[k, w[m][n]] + beta[w[m][n]])/(C[k] + sum(beta)))
            p /= sum(p)
            z[m][n] = np.nonzero(np.random.multinomial(1, p))[0][0]
            A[m, z[m][n]] += 1
            B[z[m][n], w[m][n]] += 1
            C[z[m][n]] += 1

In [15]:
theta = np.zeros((M, K))
for m in range(M):
    for k in range(K):
        theta[m , k] = (A[m, k] + alpha[k])/((sum(A[m,:])) + sum(alpha))
np.round(theta, 2)

array([[ 0.46,  0.28,  0.26],
       [ 0.64,  0.17,  0.2 ]])

In [16]:
np.round(theta_true, 2)

array([[ 0.83,  0.04,  0.13],
       [ 0.71,  0.11,  0.19]])

In [17]:
# start gibbs sampler
# num_iter = 1000
# for i in range(num_iter):
#     for m in range(M):
#         for n in range(doc_lens[m]):
#             A[m, z[m][n]] -= 1  # Decrement N1
#             B[z[m][n], w[m][n]] -= 1 # Decrement N2
#             C[z[m][n]] -= 1 # Decrement N3
#             p = np.zeros(K)
#             for k in range(K):
#                 p[k] = (A[m, k] + alpha[k])*((B[k, w[m][n]] + beta[w[m][n]])/(C[k] + sum(beta)))
#             p /= sum(p) # This is actually doing k divisions... might be a modest speed up but we can parallelize easily with numba
#             z[m][n] = np.nonzero(np.random.multinomial(1, p))[0][0]
#             A[m, z[m][n]] += 1 # Increment N1
#             B[z[m][n], w[m][n]] += 1 # Increment N2
#             C[z[m][n]] += 1 # Increment N3

In [18]:

#     initialize();
#     std::vector<double> dis((size_t)num_topics, 0);

#     CycleTimer timer;
#     for(int iter = 0; iter < num_iterations; iter++){
#         for(int d = 0; d < (int) W.size(); d++){
#             for(int j = 0; j < (int) W[d].size(); j++){
#                 int word = W[d][j];
#                 int topic = T[d][j];
#                 // ignore current position
#                 doc_topic_table[d][topic] --;
#                 topic_word_table[topic][word] --;
#                 topic_table[topic] --;

#                 // recalculate topic distribution
#                 for(int k = 0; k < num_topics; k++) {
#                     dis[k] = (topic_word_table[k][word] + beta) / (topic_table[k] + beta * vocab_size) * (doc_topic_table[d][k] + alpha);
#                 }

#                 topic = resample(dis);
#                 T[d][j] = topic;
#                 doc_topic_table[d][topic] ++;
#                 topic_word_table[topic][word] ++;
#                 topic_table[topic] ++;

#             }
#         }
#         double llh = getLogLikelihood();
#         LOG("Iteration: %d, loglikelihood: %.8f, time: %.2fs\n", iter, llh, timer.get_time_elapsed());
#     }
